# 🫁 Pneumothorax AI — Global Pre-training (Kaggle)
**TÜBİTAK 2209-A | Ahmet Demir | Dokuz Eylül Üniversitesi**

**Gereksinimler:** Settings → Accelerator: **GPU P100** | Input: SIIM-ACR | Internet: ON

In [ ]:
import torch, os, subprocess, glob, shutil, json
print('GPU :', torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'YOK — Settings > GPU P100 seç!')
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader 2>/dev/null || true

In [ ]:
# SIIM verisini nerede olursa olsun bul
print('=== /kaggle/input içeriği ===')
for root, dirs, files in os.walk('/kaggle/input', topdown=True):
    depth = root.replace('/kaggle/input', '').count('/')
    print('  ' * depth + os.path.basename(root) + '/')
    if depth >= 2:
        dirs.clear()

SIIM_INPUT = None

# 1) Önce RLE CSV'yi bul — dataset root onun yanında
rle_files = glob.glob('/kaggle/input/**/*rle*.csv', recursive=True)
if rle_files:
    SIIM_INPUT = os.path.dirname(rle_files[0])
    print(f'\n✓ SIIM bulundu (RLE): {SIIM_INPUT}')
else:
    # 2) dicom-images-train klasörünü ara
    dcm_dirs = glob.glob('/kaggle/input/**/dicom-images-train', recursive=True)
    if dcm_dirs:
        SIIM_INPUT = os.path.dirname(dcm_dirs[0])
        print(f'\n✓ SIIM bulundu (dicom-images-train): {SIIM_INPUT}')
    else:
        # 3) Herhangi bir .dcm dosyasından üst dizini çıkar
        dcm_files = glob.glob('/kaggle/input/**/*.dcm', recursive=True)
        if dcm_files:
            # DCM dosyasının bulunduğu klasörden yukarı çıkarak CSV ara
            dcm_dir = os.path.dirname(dcm_files[0])
            candidate = dcm_dir
            for _ in range(4):
                if glob.glob(f'{candidate}/*.csv'):
                    SIIM_INPUT = candidate
                    break
                parent = os.path.dirname(candidate)
                if parent in ('/kaggle/input', '/kaggle', '/'):
                    SIIM_INPUT = dcm_dir
                    break
                candidate = parent
            if SIIM_INPUT is None:
                SIIM_INPUT = dcm_dir
            print(f'✓ SIIM (dcm ile) bulundu: {SIIM_INPUT}')
        else:
            print('[!] SIIM verisi bulunamadı!')

print(f'SIIM_INPUT = {SIIM_INPUT}')

In [ ]:
REPO = 'https://github.com/ahmetai-cell/pneumothorax-ai-detection'
PROJECT_DIR = '/kaggle/working/pneumothorax-ai-detection'

if os.path.exists(PROJECT_DIR):
    result = subprocess.run('git pull', shell=True, cwd=PROJECT_DIR, capture_output=True, text=True)
    print(result.stdout)
else:
    subprocess.run(f'git clone {REPO}', shell=True, cwd='/kaggle/working')

os.chdir(PROJECT_DIR)
print('✓ Çalışma dizini:', os.getcwd())

In [ ]:
subprocess.run('pip install -q segmentation-models-pytorch albumentations pydicom pynrrd wandb tqdm fpdf2 plotly', shell=True)
print('✓ Bağımlılıklar tamam')

In [ ]:
# Dizinleri oluştur
for d in ['data/raw/global', 'data/processed', 'data/masks/siim', 'checkpoints', 'results']:
    os.makedirs(d, exist_ok=True)

SIIM_LOCAL = 'data/raw/global/siim'

if SIIM_INPUT:
    if os.path.islink(SIIM_LOCAL): os.unlink(SIIM_LOCAL)
    if os.path.isdir(SIIM_LOCAL) and not os.path.islink(SIIM_LOCAL): shutil.rmtree(SIIM_LOCAL)
    os.symlink(SIIM_INPUT, SIIM_LOCAL)
    dcm_count = int(subprocess.check_output(f'find "{SIIM_INPUT}" -name "*.dcm" | wc -l', shell=True).decode().strip())
    print(f'✓ Symlink: {SIIM_LOCAL} → {SIIM_INPUT}')
    print(f'  DICOM sayısı: {dcm_count:,}')
    
    # RLE CSV'yi bul — stage_2_train.csv veya *rle*.csv
    rle_files = (
        glob.glob(f'{SIIM_INPUT}/**/*rle*.csv', recursive=True) +
        glob.glob(f'{SIIM_INPUT}/*rle*.csv') +
        glob.glob(f'{SIIM_INPUT}/**/stage_2_train.csv', recursive=True) +
        glob.glob(f'{SIIM_INPUT}/stage_2_train.csv')
    )
    if rle_files:
        dst = f'{SIIM_LOCAL}/train-rle.csv'
        if not os.path.exists(dst):
            shutil.copy(rle_files[0], dst)
            print(f'  RLE CSV kopyalandı: {os.path.basename(rle_files[0])} → train-rle.csv')
        else:
            print(f'  RLE CSV: train-rle.csv (mevcut)')
    else:
        print('  [!] RLE CSV bulunamadı — içerik:', os.listdir(SIIM_INPUT))
else:
    print('[!] SIIM_INPUT None — yukarıdaki hücreyi kontrol et')

In [ ]:
!python scripts/data_manager.py --convert_rle
!python scripts/data_manager.py --build_manifest

import pandas as pd
try:
    df = pd.read_csv('data/processed/master_manifest.csv')
    print(f'\n✓ Manifest: {len(df):,} kayıt')
    print(f'  Pozitif : {(df["is_pneumo"]==1).sum():,}')
    print(f'  Negatif : {(df["is_pneumo"]==0).sum():,}')
    print(f'  Kaynaklar: {df["source"].value_counts().to_dict()}')
except Exception as e:
    print(f'[!] {e}')

In [ ]:
WANDB_KEY = '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'
import wandb
wandb.login(key=WANDB_KEY, relogin=True)
USE_WANDB = True
print('✓ W&B tamam')

In [ ]:
CKPT_DIR = '/kaggle/working/checkpoints'
os.makedirs(CKPT_DIR, exist_ok=True)
WANDB_FLAG = '' if USE_WANDB else '--no_wandb'

!python scripts/train_global.py \
    --sources SIIM \
    --encoder efficientnet-b0 \
    --img_size 512 \
    --epochs 50 \
    --batch_size 16 \
    --num_folds 5 \
    --lr 1e-4 \
    {WANDB_FLAG} \
    --checkpoint_dir {CKPT_DIR}

In [ ]:
for p in [f'{CKPT_DIR}/global_kfold_results.csv', 'results/global_kfold_results.csv']:
    if os.path.exists(p):
        df = pd.read_csv(p)
        print('=== K-FOLD SONUÇLARI ===')
        print(df.to_string(index=False))
        print(f'Ortalama Dice: {df["best_dice"].mean():.4f} ± {df["best_dice"].std():.4f}')
        break

mp = f'{CKPT_DIR}/global_base_model_meta.json'
if os.path.exists(mp):
    meta = json.load(open(mp))
    print('\n=== BASE MODEL ===')
    for k, v in meta.items(): print(f'  {k}: {v}')

ckpts = glob.glob(f'{CKPT_DIR}/**/*.pth', recursive=True)
print(f'\nCheckpoint sayısı: {len(ckpts)}')
for c in sorted(ckpts):
    print(f'  {os.path.basename(c):40s}  {os.path.getsize(c)/1e6:.1f} MB')

In [ ]:
OUTPUT = '/kaggle/working'
for src, dst in [
    (f'{CKPT_DIR}/global_base_model.pth',      f'{OUTPUT}/global_base_model.pth'),
    (f'{CKPT_DIR}/global_base_model_meta.json', f'{OUTPUT}/global_base_model_meta.json'),
    (f'{CKPT_DIR}/global_kfold_results.csv',    f'{OUTPUT}/global_kfold_results.csv'),
]:
    if os.path.exists(src):
        shutil.copy(src, dst)
        print(f'✓ {os.path.basename(src)}')

for ckpt in glob.glob(f'{CKPT_DIR}/global_folds/*.pth'):
    shutil.copy(ckpt, f'{OUTPUT}/{os.path.basename(ckpt)}')
    print(f'✓ {os.path.basename(ckpt)}')

print('\n✓ Output sekmesinden indirebilirsin')